In [1]:
import os

import pandas as pd
import numpy as np

from tqdm import tqdm

from deepface import DeepFace

# Process Data

In [2]:
enroll_df = pd.read_csv('../app/data/enroll_imgs_lfw.csv')
print(enroll_df.shape)
display(enroll_df.head())
print(enroll_df.gcp_path.values[0])

(1680, 3)


,person_name,img_name,gcp_path
0,Abdel_Nasser_Assidi,Abdel_Nasser_Assidi_0001.jpg,gs://original-datasets-pesquisa/face/LFW/lfw/l...
1,Abdullah_Gul,Abdullah_Gul_0012.jpg,gs://original-datasets-pesquisa/face/LFW/lfw/l...
2,Abdullah_al-Attiyah,Abdullah_al-Attiyah_0001.jpg,gs://original-datasets-pesquisa/face/LFW/lfw/l...
3,Adel_Al-Jubeir,Adel_Al-Jubeir_0002.jpg,gs://original-datasets-pesquisa/face/LFW/lfw/l...
4,Ai_Sugiyama,Ai_Sugiyama_0003.jpg,gs://original-datasets-pesquisa/face/LFW/lfw/l...


gs://original-datasets-pesquisa/face/LFW/lfw/lfw/Abdel_Nasser_Assidi/Abdel_Nasser_Assidi_0001.jpg


# Transform GCP_Paths in Paths

In [3]:
BASE_PATH = '/home/guilherme/projects/parai/app/data/'

enroll_df['img_path'] = enroll_df['gcp_path'].apply(lambda x: x.replace('gs://original-datasets-pesquisa/face/LFW/lfw/', 'images/'))
enroll_df['img_path'] = BASE_PATH + enroll_df['img_path']

enroll_df.head()

,person_name,img_name,gcp_path,img_path
0,Abdel_Nasser_Assidi,Abdel_Nasser_Assidi_0001.jpg,gs://original-datasets-pesquisa/face/LFW/lfw/l...,/home/guilherme/projects/parai/app/data/images...
1,Abdullah_Gul,Abdullah_Gul_0012.jpg,gs://original-datasets-pesquisa/face/LFW/lfw/l...,/home/guilherme/projects/parai/app/data/images...
2,Abdullah_al-Attiyah,Abdullah_al-Attiyah_0001.jpg,gs://original-datasets-pesquisa/face/LFW/lfw/l...,/home/guilherme/projects/parai/app/data/images...
3,Adel_Al-Jubeir,Adel_Al-Jubeir_0002.jpg,gs://original-datasets-pesquisa/face/LFW/lfw/l...,/home/guilherme/projects/parai/app/data/images...
4,Ai_Sugiyama,Ai_Sugiyama_0003.jpg,gs://original-datasets-pesquisa/face/LFW/lfw/l...,/home/guilherme/projects/parai/app/data/images...


# Enroll subjects from LFW dataset

In [4]:
def enroll_subject_with_deepface(img_path, fr_model):
    img_name = img_path.split('/')[-1]
    img_name = img_name.replace('.jpg', '.npy')
    template_path = f'../app/data/templates/df_templates_dlib/{img_name}'
    
    if os.path.exists(template_path):
        #print(f'{template_path} already exists')
        return
    
    template = DeepFace.represent(img_path = img_path, model=fr_model, enforce_detection=False)
    
    np.save(template_path, template)

In [4]:
extracted_encodings = os.listdir('../app/data/templates/df_templates_dlib/')

dlib_model = DeepFace.build_model('Dlib')

for input_img_path in tqdm(enroll_df['img_path'].values):
    enroll_subject_with_deepface(input_img_path, dlib_model)

100%|██████████| 1680/1680 [00:25<00:00, 65.50it/s]


In [17]:
print(f'Number of extracted templates: {len(os.listdir("../app/data/templates/"))}')

Number of extracted templates: 1556


# Enroll especific subjects 

In [6]:
fr_model = DeepFace.build_model('Dlib')

enroll_subject_with_deepface('../app/data/images/common/guilherme_1.jpg', fr_model=fr_model)
enroll_subject_with_deepface('../app/data/images/common/guilherme_2.jpg', fr_model=fr_model)
enroll_subject_with_deepface('../app/data/images/common/guilherme_3.jpg', fr_model=fr_model)
enroll_subject_with_deepface('../app/data/images/common/guilherme_4.jpg', fr_model=fr_model)